### Load the dataset

In [40]:
import numpy as np
import pandas as pd

In [41]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.columns = [_.lower() for _ in train.columns]
test.columns = [_.lower() for _ in test.columns]
print('train.shape: {}'.format(train.shape))
print('test.shape: {}'.format(test.shape))

train.shape: (26729, 10)
test.shape: (11456, 8)


### Fill in missing values & Feature engineering

#### Drop the useless attribute outcomesubtype

In [42]:
train = train.drop('outcomesubtype', axis=1)

#### Combine the training and testing data for easy processing

In [43]:
tmp_train = train.drop(['animalid', 'outcometype'], axis=1)
tmp_test = test.drop('id', axis=1)
sep = tmp_train.shape[0]
combine = pd.concat([tmp_train, tmp_test], axis=0)
print('combine.shape: {}'.format(combine.shape))

combine.shape: (38185, 7)


In [44]:
combine.head()

,name,datetime,animaltype,sexuponoutcome,ageuponoutcome,breed,color
0,Hambone,2014-02-12 18:22:00,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,Emily,2013-10-13 12:44:00,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,Pearce,2015-01-31 12:28:00,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,NaN,2014-07-11 19:09:00,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,NaN,2013-11-15 12:52:00,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


#### Create the hasname attribute and drop the original name

In [45]:
combine['hasname'] = combine.name.notnull().astype(int)
combine = combine.drop('name', axis=1)

#### Get the hour attribute and drop the original datetime

In [46]:
combine['hour'] = pd.to_datetime(combine.datetime).dt.hour.astype('category')
combine = combine.drop('datetime', axis=1)

#### Use 'intact' to replace 'sexuponoutcome'

In [47]:
combine['intact'] = combine.sexuponoutcome.fillna('Unknown').map({
    'Neutered Male': 0,
    'Spayed Female': 0,
    'Intact Male': 1,
    'Intact Female': 1,
    'Unknown': 1
})
combine = combine.drop('sexuponoutcome', axis=1)

#### Transfer animaltype into a binary attribute, use 1 to represent dog

In [48]:
combine['animaltype'] = combine.animaltype.apply(lambda x: int(x=='Dog'))

#### Transfer sexuponoutcome into days and fill in the missing values

In [49]:
def age_transfer(age_str):
    num = age_str.split(' ')[0]
    num = int(num)
    if 'day' in age_str:
        return num
    elif 'week' in age_str:
        return num * 7
    elif 'month' in age_str:
        return num * 30
    elif 'year' in age_str:
        return num * 365
    else:
        return np.nan
    
combine['ageuponoutcome'] = combine.ageuponoutcome.fillna('0').apply(age_transfer)
combine['ageuponoutcome'] = combine.groupby(['animaltype', 'intact']).ageuponoutcome.transform(lambda x: x.fillna(x.median()))

#### Extract some attributes from breed

In [59]:
# Hair
combine['hair'] = combine.breed.str.extract(r'(\w+)\s?[hH]air', expand=False).fillna('Unclear').astype('category')
# Domestic
combine['domestic'] = combine.breed.str.contains('Domestic').astype(int)

combine = combine.drop('breed', axis=1)

#### Create attribute 'morethanonecolor' from color and drop color attribute

In [60]:
combine['morethanonecolor'] = combine.color.str.contains('/').astype(int)

combine = combine.drop('color', axis=1)

#### Turn categorical attributes into dummy variables

In [62]:
combine = pd.get_dummies(combine)

In [63]:
combine.head()

,animaltype,ageuponoutcome,hasname,intact,domestic,morethanonecolor,hour_0,hour_3,hour_5,hour_6,...,hour_20,hour_21,hour_22,hour_23,hair_Long,hair_Medium,hair_Mexican,hair_Short,hair_Unclear,hair_Wire
0,1,365.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,365.0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,730.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,21.0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,730.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### Prepare the data for modeling

In [68]:
X_train = combine[:sep]
y_train = train['outcometype']
X_test = combine[sep:]

In [75]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [80]:
params = {
    'n_estimators': [100],
    'max_depth': np.arange(8, 15, 1),
    'min_samples_leaf': np.arange(2, 5, 1),
    'random_state': [0]
}

grid = GridSearchCV(RandomForestClassifier(), params, cv=8, scoring='neg_log_loss', verbose=True, n_jobs=4).fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_params_)

Fitting 8 folds for each of 21 candidates, totalling 168 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   43.7s
[Parallel(n_jobs=4)]: Done 168 out of 168 | elapsed:  3.3min finished


-0.8053171780562867
{'max_depth': 14, 'min_samples_leaf': 2, 'n_estimators': 100, 'random_state': 0}


In [74]:
cross_val_score(RandomForestClassifier(n_estimators=100), X_train, y_train, cv=8, scoring='neg_log_loss', n_jobs=4)

array([-1.86289679, -1.9117706 , -1.81616249, -1.76851657, -1.84565785,
       -1.87861655, -1.82384547, -1.81596247])